In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path
import sys
sys.path.append("../..") # go to parent dir

In [5]:
from aspects.analysis.statistics_bing_liu import (load_all_datasets)
from aspects.preprocessing.preprocesser import Preprocesser

In [16]:
import matplotlib.pyplot as plt
import pandas as pd

from typing import List, Iterable
from tqdm import tqdm
from datetime import datetime

from nltk.corpus import wordnet
from wordfreq import zipf_frequency

import spacy

from tqdm import tnrange, tqdm_notebook
import jellyfish
%matplotlib inline

In [7]:
nlp = spacy.load('en')

In [8]:
processer = Preprocesser()

# Get datasets and load 

In [9]:
datasets = load_all_datasets()

In [10]:
# similar_aspects = {
#     dataset_name: get_most_similar_text_pairs(dataset.aspect.tolist())
#     for dataset_name, dataset in datasets.items()
# }
# df = pd.DataFrame.from_dict(similar_aspects)
# df.to_csv('similar_aspects_bing_liu_datasets.csv')
# for dataset_name, similarity_scores in similar_aspects.items():
#     print(sorted([(score, pair) for pair, score in similarity_scores.items()], reverse=True))

In [11]:
df = pd.read_csv('similar_aspects_bing_liu_datasets.csv')

In [12]:
df.sample(10)

,,Canon PowerShot SD500.txt,Canon S100.txt,Diaper Champ.txt,Hitachi router.txt,Linksys Router.txt,MicroMP3.txt,Nokia 6600.txt,ipod.txt,norton.txt
195,install ## nstall ...,,,,,,,,,0.841270
486,touch control ## touch screen ...,,,,,,0.8662393162393163,,,NaN
375,reinstall ## nstall ...,,,,,,,,,0.777778
69,camera ## on - camera control ...,,0.7777777777777777,,,,,,,NaN
534,video recorder ## video ...,,,,,,,0.8714285714285714,,NaN
30,autofocus ## auto - focus ...,,0.98,,,,,,,NaN
193,instal ## uninstallation ...,,,,,,,,,0.753968
506,uninstall ## nstall ...,,,,,,,,,0.777778
105,control ## cost ...,,,,0.6761904761904761,,0.7809523809523811,,,NaN
182,hold ## hole ...,,,0.8833333333333334,,,,,,NaN


# Wordnet and zipf

In [13]:
zipf_aspects_df = {
    dataset_name: pd.DataFrame([
        (word, zipf_frequency(word, 'en')) for word in set(dataset.aspect.tolist())], 
        columns=['word', 'zipf'])
    for dataset_name, dataset in datasets.items()
} 

In [14]:
for col in zipf_aspects_df:
    print(zipf_aspects_df[col].sort_values('zipf'))

                               word  zipf
28                  beltclip design  0.00
130                         labling  0.00
149                         srorage  0.00
55                        sensitivy  0.00
191                       scrollpad  0.00
74                      batterylife  0.00
123                          plaeyr  0.00
162                      ease ofuse  0.00
148                        trnasfer  0.00
195  audible.com book compatibility  1.26
134           additional accesories  1.44
178                        hardcase  1.58
156                   work with wmp  1.58
12                 subscription wma  2.02
90                         play wma  2.03
24                        wma files  2.03
21                        backlight  2.21
211               touchpad controls  2.27
164                  touchpad light  2.27
170                        touchpad  2.27
25                       wav format  2.52
31                          earbuds  2.62
121     napster to go compatibilit

In [20]:
for synsets in wordnet.synsets('reading', pos='n'):
    print(synsets)

Synset('reading.n.01')
Synset('reading.n.02')
Synset('reading.n.03')
Synset('reading.n.04')
Synset('interpretation.n.01')
Synset('reading.n.06')
Synset('recitation.n.02')
Synset('reading.n.08')
